In [1]:
import os 
import test_module
from collections import namedtuple
import numpy as np
import retrieval_run
import settings
import cloud_dic
import utils
import test_module


In [2]:
#instrument

fwhm=-1
wavelength_range=[1,2.8]
ndata=3
Rfile='nonuniformR.dat'



#retrieval_params

##gas
chemeq=0
# gaslist = ['h2o','co','tio','vo','crh','feh','na','k']
gaslist = ['h2o','co','tio','vo','crh','feh','k','na']
gastype_list=['U','U','U','U','U','N','N','N']
ptype=77  

## clouds

do_clouds=1
cloudpath='/Users/fw23aao/Downloads/cloud_pickles/'

# npatches=1
# cloud_name=['clear','powerlaw', 'grey', 'tholin_WS15.mieff']
# cloud_type=['None',"deck","slab","deck"]
# cloudpatch_index=[[1],[1],[1],[1]]
# particle_dis=['','', '', 'hansen']


npatches=2
cloud_name=['powerlaw', 'grey', 'ZnS_WS15.mieff', 'tholin_WS15.mieff']
cloud_type=["Deck","slab","deck","deck"]

cloudpatch_index=[[1,2],[1],[1,2],[1]]
particle_dis=['None', 'None', 'hansen',"log_normal"]


# ModelConfig:
do_fudge=1
samplemode='mcmc'

# instrument_instance = utils_dev.Instrument(fwhm,wavelength_range,ndata,Rfile)
# re_params = utils_dev.Retrieval_params(samplemode,chemeq,gaslist,gastype_list,fwhm,do_fudge,ptype,do_clouds,npatches,cloudname,cloudpacth_index,particle_dis)
# model_config_instance = utils_dev.ModelConfig(samplemode,do_fudge)
# io_config_instance = utils_dev.IOConfig()


instrument_instance = utils.Instrument(fwhm,wavelength_range,ndata,Rfile)
re_params = utils.Retrieval_params(samplemode,chemeq,gaslist,gastype_list,fwhm,do_fudge,ptype,do_clouds,npatches,cloud_name,cloud_type,cloudpatch_index,particle_dis)
model_config_instance = utils.ModelConfig(samplemode,do_fudge,cloudpath=cloudpath)
io_config_instance = utils.IOConfig()





model_config_instance.dist= 5.84
model_config_instance.xlist ='data/gaslistRox.dat'
model_config_instance.do_bff=0
model_config_instance.malk=1
model_config_instance.ch4=0
model_config_instance.update_dictionary()


In [3]:
obspec = np.asfortranarray(np.loadtxt("./examples/example_data/G570D_2MHcalib.dat",dtype='d',unpack='true')) # G570D_2MassJcalib.dat
args_instance = utils.ArgsGen(re_params,model_config_instance,instrument_instance,obspec)
settings.init(args_instance)

In [4]:
args_instance

In [5]:
np.shape(args_instance.cloudname_set),args_instance.cloudname_set

((4,),
 ['powerlaw cloud deck',
  'grey cloud slab',
  'Mie scattering cloud deck--ZnS_WS15.mieff',
  'Mie scattering cloud deck--tholin_WS15.mieff'])

In [6]:
np.shape(args_instance.cloud_opaname),args_instance.cloud_opaname

((4,),
 array(['powerlaw', 'grey', 'ZnS_WS15.mieff', 'tholin_WS15.mieff'],
       dtype=object))

In [7]:
np.shape(args_instance.cloudmap),args_instance.cloudmap

((2, 4),
 array([[ True,  True,  True,  True],
        [ True, False,  True, False]]))

In [8]:
np.shape(args_instance.cloudsize),args_instance.cloudsize

((4,), array([0, 0, 1, 2]))

In [9]:
np.shape(args_instance.cloudata)

(4, 3, 196, 60)

In [10]:
args_instance=settings.runargs
all_params,all_params_values =utils.get_all_parametres(re_params.dictionary)

if re_params.samplemode=='mcmc':
    
    model_config_instance.ndim=len(all_params)
    model_config_instance.nwalkers=len(all_params)*16

    r2d2 = (71492e3)**2. / (model_config_instance.dist * 3.086e+16)**2.
    re_params.dictionary['refinement_params']['params']['r2d2']['distribution']=['normal',r2d2,0.1*r2d2]


    def tolerance_parameter_customized_distribution(x):
        return np.log10((np.random.rand(x)* (max(args_instance.obspec[2,:]**2)*(0.1 - 0.01))) + (0.01*min(args_instance.obspec[2,10::3]**2))) 

    for i in range(len(all_params)):
        if all_params[i].startswith('tolerance_parameter'):
            re_params.dictionary['refinement_params']['params'][all_params[i]]['distribution']=['customized',tolerance_parameter_customized_distribution]

    if model_config_instance.fresh == 0:
        p0=utils.MC_P0_gen(re_params.dictionary,model_config_instance,args_instance)

In [11]:
params_master = namedtuple('params',all_params)
params_instance = params_master(*p0[0,:])
params_instance

params(h2o=-4.8312926120582365, co=-4.177137910079509, tio=-3.5825150406691435, vo=-4.048362738466376, crh=-4.18244235136604, feh=-4.4810246179787185, p_ref_feh=-0.8122990618924251, alpha_feh=0.5265587804124237, K_Na=-3.2448592078253724, p_ref_K_Na=-0.833509431096836, alpha_K_Na=0.8597993234103277, logg=4.537650853746592, r2d2=1.6919355130992372e-19, scale1=0.0002625009066124352, scale2=-0.0003215949833731092, dlambda=4.0187665398335044e-05, tolerance_parameter_1=-32.30534046815328, tolerance_parameter_2=-32.21385771957188, tolerance_parameter_3=-32.52640164304063, gamma=50.269916605468275, Tint=1196.5880573655731, alpha=1.320531178463671, lndelta=0.46253740176717856, T1=1029.711765911393, T2=1286.461211819721, T3=1435.1894593577908, fcld=0.6501387485218293, logp_pcd=-0.1040586607350866, dp_pcd=0.023546543945320072, omega_pcd=0.8034856296999977, alpha_pcd=2.7967368294342503, tau_gcs=7.560310999439254, logp_gcs=-0.05698227886747903, dp_gcs=0.03139352937498745, omega_gcs=0.67641998706000

In [12]:
import cloud_dic_new


cloudparams=cloud_dic_new.cloud_unpack(re_params,params_instance)
cloudprof,cloudrad,cloudsig = cloud_dic_new.atlas(re_params,cloudparams,args_instance.press)

In [13]:
np.shape(cloudparams),cloudparams

((5, 4),
 array([[ 0.        ,  7.560311  ,  0.        ,  0.        ],
        [-0.10405866, -0.05698228,  1.11066068,  0.96212707],
        [ 0.02354654,  0.03139353,  0.08601376,  0.01692693],
        [ 0.80348563,  0.67641999, -1.27228041, -1.40537585],
        [ 2.79673683,  0.        ,  0.22166489, -1.01844333]]))

In [14]:
np.shape(cloudprof)

(64, 4)

In [15]:
np.shape(cloudrad)

(64, 4)

In [16]:
np.shape(cloudsig)

(64, 4)

In [17]:
cloudprof = np.asfortranarray(cloudprof,dtype = 'float64')
cloudrad = np.asfortranarray(cloudrad,dtype = 'float64')
cloudsig = np.asfortranarray(cloudsig,dtype = 'float64')

In [18]:
print(args_instance.cloud_opaname.flags['F_CONTIGUOUS'])
print(args_instance.cloudata.flags['F_CONTIGUOUS']) 
print(args_instance.miewave.flags['F_CONTIGUOUS']) 
print(args_instance.mierad.flags['F_CONTIGUOUS'])
print(args_instance.cloudsize.flags['F_CONTIGUOUS'])
print(args_instance.cloudmap.flags['F_CONTIGUOUS'])
print(cloudprof.flags['F_CONTIGUOUS'])
print(cloudrad.flags['F_CONTIGUOUS'])
print(cloudsig.flags['F_CONTIGUOUS'])

True
True
True
True
True
True
True
True
True


In [19]:
# gnostics = 0
# shiftspec, photspec, tauspec,cfunc = test_module.modelspec(p0[0,:],re_params,args_instance,gnostics)

In [52]:

pattern_opa = re.compile(r'\b(powerlaw|grey|Mie)\b', re.IGNORECASE)

cloud_opatype=[]
for idx, name in enumerate(cloudname_set, start=1):
    match = pattern_opa.search(name)
    cloud_opatype.append(match.group(1).lower() if match else 'unknown')

In [53]:
cloud_opatype

['powerlaw', 'grey', 'mie']